In [0]:
#Installation of tesseract - the main tool for text extraction
!sudo add-apt-repository ppa:alex-p/tesseract-ocr-devel -y
!sudo apt-get update
!apt install tesseract-ocr
!apt install libtesseract-dev
!pip install pytesseract
!pip install imantics

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/alex-p/tesseract-ocr-devel/ubuntu bionic InRelease [15.4 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:12 https://developer.

In [0]:
!pip install python-Levenshtein

     |████████████████████████████████| 51kB 2.3MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144796 sha256=07b3494fe1ba1f193a3f8e8661d76735dd675545dca05f1403fd98d50c2a7b81
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein


In [0]:
#importing tools for data visualisation, math operations with matrices and tensors(Keras, numpy), and text extraction(pytesseract)
from google.colab import drive
from keras import utils
import keras_preprocessing.image.utils as image_utils
from PIL import Image, ImageDraw
from imantics import Mask
import glob
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pytesseract
import pandas as pd
import csv
from Levenshtein import *
drive.mount("/content/drive", force_remount = True)


WORKDIR = "/content/drive/My Drive/Colab Notebooks/Materials/Text extraction"

Mounted at /content/drive


In [0]:
def getCvImage(name):
  gray = cv2.imread(name, cv2.IMREAD_GRAYSCALE)
  for i in range(gray.shape[0]):
    for j in range(gray.shape[1]):
      gray[i][j] = 255 - gray[i][j]
  return gray

def getPilImage(name):
  img = Image.open(name).convert('LA')
  arr = np.array(img)
  for i in range(len(arr)):
    for j in range(len(arr[i])):
      arr[i][j][0] = 255 - arr[i][j][0]
  img = Image.fromarray(arr)
  return img

In [0]:
reader = csv.DictReader(open("/content/drive/My Drive/Colab Notebooks/Materials/Text extraction/csv/tesseract_dataset.csv"))
scores = []
for raw in reader:
    dictt = {}
    strin = list(raw['image_name;team_score1;team_score2'].split(';'))
    dictt['image_name'] = strin[0]
    dictt['team_score1'] = strin[1]
    dictt['team_score2'] = strin[2]
    scores.append(dictt)

In [0]:
empty = []
normal = []
counter = 0
gs_counter = 0
all_len = 0
plt.figure(figsize=(30,10))
z=0
for filename in glob.glob(WORKDIR + '/datasets/tesseract_score/train/*.png'):
  img1 = getCvImage(filename)
  img2 = getPilImage(filename)
  pred1 = pytesseract.image_to_string(img1, config = '--psm 6 -c tessedit_char_whitelist="0123456789"')
  pred2 = pytesseract.image_to_string(img2, config = '--psm 6 -c tessedit_char_whitelist="0123456789"')
  st = list(filename.split("/"))
  if pred1 == '' or pred2 == '':
    empty.append(st[-2] + "/" +st[-1])
  st = list(filename.split("/"))
  name = st[-1].split("_")
  for score in scores:
    if score['image_name'] == name[1]:
      if name[0] == '1' and pred1 == score['team_score1']:
        #counter+=1
        normal.append(st[-2] + "/" +st[-1])
      elif name[0] == '2' and pred1 == score['team_score2']:
        #counter+=1
        normal.append(st[-2] + "/" +st[-1])

plt.show()
dist = 0
gs_dist = 0
for filename in glob.glob(WORKDIR + '/datasets/tessaract_score/test/*.png'):
  img1 = getCvImage(filename)
  pred1 = pytesseract.image_to_string(img1, config = '--psm 6 -c tessedit_char_whitelist="0123456789"')
  pred2 = pytesseract.image_to_string(img1, lang="game_score", config = '--psm 6 -c tessedit_char_whitelist="0123456789"')
  st = list(filename.split("/"))
  if pred1 == '' or pred2 == '':
    empty.append(st[-2] + "/" +st[-1])
  st = list(filename.split("/"))
  name = st[-1].split("_")
  for score in scores:
    if score['image_name'] == name[1]:
      if name[0] == '1':
        all_len+=len(score["team_score1"])
        dist+=distance(pred1, score['team_score1'])
        if distance(pred1, score['team_score1']) == 0:
          counter+=1
        gs_dist+=distance(pred2, score['team_score1'])
        if distance(pred2, score['team_score1']) == 0:
          gs_counter+=1
        normal.append(st[-2] + "/" +st[-1])
      elif name[0] == '2':
        all_len+=len(score["team_score2"])
        dist+=distance(pred1, score['team_score2'])
        if distance(pred1, score['team_score2']) == 0:
          counter+=1
        gs_dist+=distance(pred2, score['team_score2'])
        if distance(pred2, score['team_score2']) == 0:
          gs_counter+=1
        normal.append(st[-2] + "/" +st[-1])



for filename in glob.glob(WORKDIR + '/datasets/tesseract_score/validation/*.png'):
  img1 = getCvImage(filename)
  pred1 = pytesseract.image_to_string(img1, config = '--psm 6 -c tessedit_char_whitelist="0123456789"')
  pred2 = pytesseract.image_to_string(img1, lang="game_score", config = '--psm 6 -c tessedit_char_whitelist="0123456789"')
  st = list(filename.split("/"))
  if pred1 == '' or pred2 == '':
    empty.append(st[-2] + "/" +st[-1])
  st = list(filename.split("/"))
  name = st[-1].split("_")
  for score in scores:
    if score['image_name'] == name[1]:
      if name[0] == '1':
        all_len+=len(score["team_score1"])
        dist+=distance(pred1, score['team_score1'])
        if distance(pred1, score['team_score1']) == 0:
          counter+=1
        gs_dist+=distance(pred2, score['team_score1'])
        if distance(pred2, score['team_score1']) == 0:
          gs_counter+=1
        normal.append(st[-2] + "/" +st[-1])
      elif name[0] == '2':
        all_len+=len(score["team_score2"])
        dist+=distance(pred1, score['team_score2'])
        if distance(pred1, score['team_score2']) == 0:
          counter+=1
        gs_dist+=distance(pred2, score['team_score2'])
        if distance(pred2, score['team_score2']) == 0:
          gs_counter+=1
        normal.append(st[-2] + "/" +st[-1])


print("Standard accuracy: ", counter/80, "Learned model accuracy: ", gs_counter/80)
print("Standard Levenshtein: ", dist, "Learned model Levenshtein: ", gs_dist)
print("Standard error: ", dist/all_len, "Learned model error: ", gs_dist/all_len)

<Figure size 2160x720 with 0 Axes>

Standard accuracy:  0.4125 Learned model accuracy:  0.95
Standard Levenshtein:  48 Learned model Levenshtein:  5
Standard error:  0.4528301886792453 Learned model error:  0.04716981132075472


In [0]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/Screen\ segmentation/datasets/tessaract2/train/blackwhite
!ls -1 *.lstmf > all-lstmf
#!tesseract 1_60.png 1_60 --psm 6 lstm.train
#!tesseract 2_50.png 2_50 --psm 6 lstm.train
#!tesseract 2_60.png 2_60 --psm 6 lstm.train
#!tesseract 2_41.png 2_41 --psm 6 lstm.train

/content/drive/My Drive/Colab Notebooks/Screen segmentation/datasets/tessaract2/train/blackwhite


In [0]:
!lstmtraining \
  --continue_from /content/drive/My\ Drive/Colab\ Notebooks/Screen\ segmentation/datasets/tessaract2/train/blackwhite/eng.lstm \
  --model_output /content/drive/My\ Drive/Colab\ Notebooks/Text\ extraction/checkpoints/chechpoint \
  --traineddata /content/drive/My\ Drive/Colab\ Notebooks/Screen\ segmentation/datasets/tessaract2/train/blackwhite/eng.traineddata \
  --train_listfile all-lstmf \
  --max_iterations 3000

In [0]:
!lstmtraining --stop_training \
  --continue_from /content/drive/My\ Drive/Colab\ Notebooks/Text\ extraction/checkpoints/chechpoint_checkpoint \
  --traineddata /content/drive/My\ Drive/Colab\ Notebooks/Screen\ segmentation/datasets/tessaract2/train/blackwhite/eng.traineddata \
  --model_output /content/drive/My\ Drive/Colab\ Notebooks/Text\ extraction/game_score.traineddata

Loaded file /content/drive/My Drive/Colab Notebooks/Text extraction/checkpoints/chechpoint_checkpoint, unpacking...


In [0]:
!cp '/content/drive/My Drive/Colab Notebooks/Materials/Text extraction/traineddata/game_score.traineddata' /usr/share/tesseract-ocr/5/tessdata/game_score.traineddata